In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate


In [3]:

# 임베딩 및 LLM (Upstage 사용 불가시 OpenAI/기타 모델 대체)
try:
    from langchain_upstage import UpstageEmbeddings, ChatUpstage
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.2
    )
except:
    from langchain_openai import OpenAIEmbeddings, ChatOpenAI
    embeddings = OpenAIEmbeddings()
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)


In [4]:

# 0단계: 문서 로드
loader = PyPDFLoader("../data/콘텐츠분쟁해결_사례.pdf")
documents = loader.load()


In [5]:

# 1단계: 문서 분할 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=300,
    separators=[
        "\n【사건개요】",
        "\n【쟁점사항】",
        "\n【처리경위】",
        "\n【처리결과】",
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)

splits = text_splitter.split_documents(documents)


In [6]:

# 2단계: 벡터스토어 생성
vectorstore = FAISS.from_documents(splits, embeddings)
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)


In [7]:

# 5단계: 프롬프트 템플릿
prompt_template = """당신은 콘텐츠 분야 전문 법률 자문가입니다.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요
2. 관련 법령이나 조항이 있다면 명시하세요
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요
4. 실무적 해결방안을 단계별로 제시하세요
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요

전문 법률 조언:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)


In [8]:

# 6단계: QA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)


In [9]:

# 7단계: 테스트 질문
questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]


In [10]:

# 8단계: 선택적 분쟁 유형 분류 함수
def classify_dispute_type(query):
    game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
    elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
    web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]

    query_lower = query.lower()

    if any(keyword in query_lower for keyword in game_keywords):
        return "게임"
    elif any(keyword in query_lower for keyword in elearning_keywords):
        return "이러닝"
    elif any(keyword in query_lower for keyword in web_keywords):
        return "웹콘텐츠"
    else:
        return "기타"


In [11]:

# 실행 예시
for q in questions:
    result = qa_chain({"query": q})
    print("질문:", q)
    print("분쟁 유형:", classify_dispute_type(q))
    print("답변:", result["result"][:500], "...\n")



질문: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
분쟁 유형: 게임
답변: ### 전문 법률 조언: 온라인 게임 시스템 오류로 인한 아이템 복구 거부 시 해결 방안  

#### 1. **사례 분석을 통한 법적 쟁점 정리**  
제시된 사례집에 따르면, 시스템 오류로 인한 아이템 복구 문제는 다음과 같은 쟁점으로 판단됩니다:  
- **시스템 오류 인정 여부**: 게임사가 오류 발생을 공식적으로 인정하거나, 다수 이용자에게 동일한 피해가 발생한 경우 복구 가능성이 높음(2006년 사례 참조).  
- **계정 소유권 문제**: 계정 명의자와 실소유자가 다를 경우, 게임사는 약관상 복구를 거부할 수 있음(2006년 사례).  
- **피신청인의 입증 책임**: 게임사는 오류가 없음을 입증해야 하며, 단순히 "통신 문제" 등으로 주장할 경우 설득력이 떨어짐(2009년 사례).  
- **아이템의 법적 성격**: 게임 아이템은 일반적으로 "전자적 데이터"로 분류되며, 민법상 재산권 인정 여부는 불분명합니다. 다만, **민법 제250조(도품·유실물 특례)**가 유 ...

질문: 인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?
분쟁 유형: 이러닝
답변: ### 전문 법률 조언: 인터넷 강의 중도 해지 시 과도한 위약금 요구 문제

#### 1. **사례 근거 분석**  
제시된 한국소비자원 조정결정례(2007~2010년)에 따르면, 인터넷 교육 서비스 중도 해지 시 과도한 위약금 요구는 대부분 **소비자분쟁해결기준** 및 **방문판매 등에 관한 법률(방판법)** 위반으로 판단되었습니다.  
- **공통 쟁점**: 위약금, 사은품 비용, 컨텐츠 비용 등 공제 항목의 적정성.  
- **결정 경향**:  
  - 실제 이용일수 + 총 이용금액의 **10% 이내 위약금**만 인정.  
  - 계약서에 명시되지 않은 비용(예: 사은품, 컨텐츠 비용)은 공제 불가. 